## Data collection

I have found a [dataset](https://github.com/antibot4navalny/accounts_labelled/blob/main/labels.json) of accounts recognized as Twitter bots'. The dataset is used in Chrome extention '[MetaBot for Twitter](https://github.com/antibot4navalny/metabot)' that highlights bots on Twitter.

To get a list of account names from the JSON I simply subset even keys.

As I have collected some data before I use list comparation to update information only for new names in bot list (updatable).

In [411]:
import pandas as pd
import json

from twitter_scraper_selenium import get_profile_details

In [10]:
try:
    df = pd.read_csv('twitter_profiles_dataset.csv')
    bots_old_names = set(df.screen_name.tolist())
except:
    bots_old_names = ()
print('bots_old_names: ', len(bots_old_names))

bots_old_names:  672


In [11]:
# Turn bots json file into a dictionary and compare with an old list of bots to check for updates:
bots_url = 'https://raw.githubusercontent.com/antibot4navalny/accounts_labelled/main/labels.json'
bots_dict = pd.read_json(bots_url, typ='series')
bots_list = list(bots_dict.keys())
bots_names = bots_list[1::2]

bots_new_names = list(set(bots_names) - set(bots_old_names))

bots_new_names
print(len(bots_new_names), bots_new_names)

51 ['GordeevaSamara', 'Katheri92373666', 'KathyHo87241217', 'kostkornl', 'Rosalva23944962', 'iraannhh', 'ilushakov_', 'lavrent_pasha', 'zokuznecova', 'Jennife46249662', 'Janell66927328', 'ded_maloletnij', 'sigazasigoy', 'AndreaP73330414', 'gordej_popov', 'skvorcov_trosha', 'IPracko', 'Tamekia66825359', 'hochu_pitsuu', 'ehidnaa_k', 'ollivimesh', 'kottowmak', 'grigfeds', 'zazerkal_v', 'ahmadiev_anton', 'apokalipsisa_s', 'aisa_letova', 'KMarmedova', 'vasencoo', 'osipovsah', 'svtojblinik_', 'licnyj_sort', 'polutonnaa', 'baskortotstan', 'marienmak', 'SofiMitrofanov_', 'oleghromov_', 'marknikoll', 'Jennife91361116', 'Jennife93704963', 'markinowa', 'tomatnyjsok2', 'justanaccsi', 'Krystal42545662', 'SasaKaramzin', 'kaztinina', 'JodiHeidenreic3', 'saharvvas', 'Misherbakov', 't_sledak', 'martinovpt']


In [15]:
# Use a simple loop for collecting bots' profiles into one dataframe:
profiles = []
for bot in bots_new_names:
    bot_profile = get_profile_details(twitter_username=bot)
    data = json.loads(bot_profile)
    profiles.append(data)
    #time.sleep(1)

df_bots_new = pd.json_normalize(profiles)

In [ ]:
# Mark bot users by '1' 
df_bots_new['bot'] = 1

In [69]:
# Append old dataset with collected new data:
try:
    df = pd.concat([df, df_bots_new], axis=0, ignore_index=True)
except:
    df = df_bots_new

df

,id,id_str,name,screen_name,location,profile_location,description,url,protected,followers_count,...,status.retweeted_status.possibly_sensitive,status.retweeted_status.possibly_sensitive_editable,status.retweeted_status.self_thread.id,status.retweeted_status.self_thread.id_str,status.retweeted_status.entities.media,status.retweeted_status.extended_entities.media,status.self_thread.id,status.self_thread.id_str,entities.url.urls,bot
0,1.582274e+18,1.582274e+18,путешествие в маусвиль,MausvilV,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1.584492e+18,1.584492e+18,Елена Горбачева,lenagrbu,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1.557303e+18,1.557303e+18,Ева Карпова,EvaKarpoff,NaN,NaN,Просто Ева ❤️,NaN,False,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1.584519e+18,1.584519e+18,Даниил Воробушкин,d_vorobuskin,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1.580882e+18,1.580882e+18,вечеринка с коктейлями и тако,SVecerinka,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [70]:
# Save our dataframe to .csv file:
df.to_csv('twitter_profiles_dataset.csv', encoding='utf-8', index=False)

Now I have a list of bot users and now it's time to mix them with real person users to get a train set for a bot classification problem.
The file "organic_names_IDs.txt" was provided by the same author.

In [71]:
organic_url = "organic_names_IDs.txt"
organic_df = pd.read_csv(organic_url, sep='\t', header=None, na_filter=False)
organic_names = list(organic_df[0].unique())
print('organic_names: ', len(organic_names))

organic_names:  551


The way of collecting organic profiles data is the same.

In [468]:
profiles = []
for organic in organic_names:
    organic_profile = get_profile_details(twitter_username=organic)
    data = json.loads(organic_profile)
    profiles.append(data)
    #time.sleep(1)

df_organic_new = pd.json_normalize(profiles)

,id,id_str,name,screen_name,location,profile_location,description,url,protected,followers_count,...,status.retweeted_status.place.bounding_box.coordinates,status.withheld_scope,status.withheld_in_countries,status.retweeted_status.withheld_scope,status.retweeted_status.withheld_in_countries,status.retweeted_status.quoted_status_id,status.retweeted_status.quoted_status_id_str,status.self_thread.id,status.self_thread.id_str,bot
0,1.510548e+18,1.510548e+18,Vladimir Vataman,VladimirVatama2,NaN,NaN,NaN,NaN,False,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1.510550e+18,1.510550e+18,Dzapo Lucci,Dimosyno,Третья от Солнца,NaN,Растите ваши крылья поближе к голове.\nВо избе...,NaN,False,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1.510732e+18,1.510732e+18,Светлана Грицкова,SGrickova,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1.510869e+18,1.510869e+18,Толендияр Сейдакбаров,tolendiar,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1.510928e+18,1.510928e+18,Петр Углов,UglovPP,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,1.583160e+18,1.583160e+18,Runner to Freedom,RunnertoFreedom,NaN,NaN,#nowar\nNo name writer,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
547,1.575539e+18,1.575539e+18,@Javi_DarkOSINT,JDarkosint,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
548,1.580157e+18,1.580157e+18,Vahmurka,admiral_sraka,NaN,NaN,NaN,NaN,False,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
549,1.582674e+18,1.582674e+18,Stellar Wildcat,WildStellarCat,NaN,NaN,NaN,NaN,False,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Mark organic users by '0' 
df_organic_new['bot'] = 0

In [ ]:
# Append old dataset with collected new data:
try:
    df = pd.concat([df, df_organic_new], axis=0, ignore_index=True)
except:
    df = df_organic_new

df

In [ ]:
# Save our dataframe to .csv file:
df.to_csv('twitter_profiles_dataset.csv', encoding='utf-8', index=False)